# Multiplicação de 2 matrices quadradas utilizando um bloco unico de therads e uma memoria global. Cada thread calcula um elemento do resultado da matriz

In [1]:
import numpy as np
from pycuda import compiler, gpuarray, tools

In [2]:
# inicializar o device
import pycuda.autoinit

Criamos nossa função de Kernel:

In [9]:
kernel_code = """
__global__ void MatrixMulKernel(float *a, float *b, float *c, int matrixSize)
{
    // Thread 2D 
    int tx = threadIdx.x;
    int ty = threadIdx.y;
    
    // Criamos Pvalue para armazenar os elementos da matrix que estão sendo computadas
    float Pvalue = 0;
    
    // Each thread loads one row of M and one column of N,
    // to produce one element of P.
    for (int k=0; k< matrixSize; ++k) {
        float Aelement = a[ty * matrixSize + k];
        float Belement = b[k * matrixSize + tx];
        Pvalue += Aelement*Belement;
    }

    //Escreve a matriz na memoria do GPU
    //Cada thread escreve um elemento
    c[ty * matrixSize + tx] = Pvalue;
}
"""

Definimos as variáveis no CPU , criamos 2 matrizes quadradas aleatorias:

In [4]:
MATRIX_SIZE=2

a_cpu = np.random.randn(MATRIX_SIZE,MATRIX_SIZE).astype(np.float32)
b_cpu = np.random.randn(MATRIX_SIZE,MATRIX_SIZE).astype(np.float32)

In [5]:
# calculamos nossa referencia multiplicando a_cpu*b_cpu para comparar com o calculo em GPU
c_cpu = np.dot(a_cpu,b_cpu)

Definimos as variáveis no GPU , transferir da memoria do host (CPU) para memoria do device (GPU):

In [6]:
a_gpu = gpuarray.to_gpu(a_cpu)
b_gpu = gpuarray.to_gpu(b_cpu)

In [7]:
# Criamos uma matriz vazia para o resultado (C = A * B)
c_gpu = gpuarray.empty((MATRIX_SIZE, MATRIX_SIZE), np.float32)

Compilamos o kernel:

In [10]:
mod = compiler.SourceModule(kernel_code)

/home/gpuuser/miniconda2/envs/envanaconda33/lib/python3.3/site-packages/ipykernel/__main__.py:1: UserWarning: The CUDA compiler succeeded, but said the following:
nvcc warning : The 'compute_20', 'sm_20', and 'sm_21' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).

  if __name__ == '__main__':


Pegamos a função do kernel desde o modulo compilado:

In [12]:
matrixmul = mod.get_function("MatrixMulKernel")

Chamamos o kernel que está no GPU:

In [15]:
matrixmul(
        # inputs
        a_gpu, b_gpu,
        # output
        c_gpu,
        # matrix_size
        np.int32(MATRIX_SIZE),
        # (só um bloco) um bloco de dimensão MATRIX_SIZE x MATRIX_SIZE threads
        block = (MATRIX_SIZE, MATRIX_SIZE,1),
        )

Apresentamos os resultados:

In [16]:
print("-" * 80)
print("Matrix A (GPU):")
print(a_gpu.get())

print("-" * 80)
print("Matrix B (GPU):")
print(b_gpu.get())

print("-" * 80)
print("Matrix C (GPU):")
print(c_gpu.get())

print("-" * 80)
print("CPU-GPU difference:")
print(c_cpu - c_gpu.get())

np.allclose(c_cpu, c_gpu.get())

--------------------------------------------------------------------------------
Matrix A (GPU):
[[-0.01832112 -0.62257814]
 [ 2.017344   -1.30206716]]
--------------------------------------------------------------------------------
Matrix B (GPU):
[[-0.05257856 -0.95913631]
 [ 1.8141917  -1.26291764]]
--------------------------------------------------------------------------------
Matrix C (GPU):
[[-1.12851286  0.80383736]
 [-2.46826839 -0.29050434]]
--------------------------------------------------------------------------------
CPU-GPU difference:
[[  1.19209290e-07   0.00000000e+00]
 [ -2.38418579e-07   0.00000000e+00]]


True